In [217]:
from openpyxl import Workbook, load_workbook
from openpyxl.writer.excel import save_virtual_workbook
import os
import pandas as pd
import numpy as np
import scipy as sp
from scipy import interpolate
from array import array
from datetime import datetime, timedelta
from pycel import ExcelCompiler
from dateutil.relativedelta import relativedelta

In [232]:
class SheetAnalyzer:

    BASE_ELEMENTS_ROW = 17 # Location of base's elements
    UNIT_ROW = 18 # Location of units
    CURVE = 19 # Location of Curve's interpolations
    METADATA_COL = "A"
    POINT_N_COL = "B"
    DATE_COL = "D"

    def __init__(self, input, sheet_number) -> None:
        self.wb = load_workbook(os.getcwd()+ input)
        self.evaluator = ExcelCompiler(filename=os.getcwd()+ input)
        self.sheet_number = sheet_number
        self.sheet_name = self.wb.sheetnames[sheet_number]
        self.ws = self.wb[self.sheet_name]

        self.specifications = []
        self.points = []
        self.json = {}

    ### Util's functions

    def log_interp1d(self, xx, yy, kind='linear'):
        logx = np.log10(xx)
        logy = np.log10(yy)
        lin_interp = interpolate.interp1d(logx, logy, kind=kind, fill_value="extrapolate")
        log_interp = lambda zz: np.power(10.0, lin_interp(np.log10(zz)))
        return log_interp

    ### Helper's functions

    def _fullFillPointsWithDates(self, points:array):
        """
            Return a fullfill array of points associate with dates 
        """
        ref_date = None
        for index, point in enumerate(points):
            if point["date"] is not None:
                ref_date = {"index": index, "date": point["date"]}
                break
        
        if ref_date is not None:
            for index, point in enumerate(points):
                if point["date"] is None:
                    if index < ref_date["index"]:
                        point["date"] = ref_date["date"] + relativedelta(years=-(ref_date["index"]-index))
                    else:
                        point["date"] = ref_date["date"] + relativedelta(years=(index-ref_date["index"]))
        
        return points
            
    def _getPointsWithDates(self):
        """
            Return dict (row_number, point_no, date) of all points and associate's date
        """
        points=[]
        for point in self.ws[self.POINT_N_COL]:
            if point.value is not None and type(point.value) is int:
                points.append({
                    "row": point.row, 
                    "point_n": point.value, 
                    "date": self.ws[self.DATE_COL+str(point.row)].value
                })
            elif point.value is not None and point.value.startswith("="):
                points.append({
                    "row": point.row, 
                    "point_n": self.evaluator.evaluate(self.sheet_name+"!"+point.coordinate), 
                    "date": self.ws[self.DATE_COL+str(point.row)].value
                })
            else:
                continue
        
        return self._fullFillPointsWithDates(points)

    def _getSpecifications(self):
        """
        Return dict (column, specification, unit, interpolation) at BASE_ELEMENTS_ROW for a given sheetname
        Ignore the firsts 2 elements because they always not belong to specifications
        """
        return [{
            "column": be.column, 
            "specification_name": be.value, 
            "unit": self.ws.cell(row=self.UNIT_ROW, column=be.column).value, 
            "interpolation": self.ws.cell(row=self.CURVE, column=be.column).value
        } for be in self.ws[self.BASE_ELEMENTS_ROW] if be.value is not None][2:]

    def _getValuesBySpecificiation(self, specification):
        result = []
        value = None
        for point in self.points:
            cell = self.ws.cell(row=point["row"], column=specification["column"])
            if cell.value is not None:
                if isinstance(cell.value, str):
                    if cell.value.startswith("="):
                        value = self.evaluator.evaluate(self.sheet_name+"!"+cell.coordinate)
                    if "%" in cell.value:
                        value = float(cell.value.replace("%", ""))/100
                elif isinstance(cell.value, float) or isinstance(cell.value, int):
                    value = cell.value
                result.append({
                    "row": point["row"],
                    "value": value
                })
        return result 

    def _addInterpolation(self, specification):
        values = self._getValuesBySpecificiation(specification)
        if specification["interpolation"] == "CONST" or (specification["interpolation"] is None and len(values) == 1):
            return [next(item["value"] for item in values if item["value"] is not None)] * len(self.points)
        else:
            interp1d = interpolate.interp1d([v["row"] for v in values], [v["value"] for v in values], fill_value="extrapolate")
            if specification["interpolation"] == "LINEAR":
                pass # Default behavior
            
            elif specification["interpolation"] == "LOG":
                interp1d = self.log_interp1([v["row"] for v in values], [v["value"] for v in values])

            result = []
            for point in self.points:
                if not any(v["row"] == point["row"] for v in values):
                    result.append(float(interp1d(point["row"])))
                else:
                    result.append(list(filter(lambda v: v["row"] == point["row"], values))[0]["value"])
            return result            

    def _getSpectificationsWithInterpolations(self):
        """
            return a dict with specifications and interpolations
        """
        result = []
        if self.points == []:
            self.points = self._getPointsWithDates()

        if self.specifications == []:
            self.specifications = self._getSpecifications()

        for index, specification in enumerate(self.specifications):
            if index == 17:
                specification["values"] = self._addInterpolation(specification)
                return specification

    def _getBaseElementMetaData(self):
        """
        Return array of tuples (row_id, metadata_name, metadata_value) for a given sheetname
        """
        return [(cell.row, cell.value, self.ws['B'][cell.row].value) for cell in self.ws[self.METADATA_COL] if (cell.value is not None and cell.row < self.BASE_ELEMENTS_ROW)]

    def _getRawDataStorage(self):
        """
            Return JSON storage of the input
        """
        return self._getSpectificationsWithInterpolations()

In [233]:
sa = SheetAnalyzer("/input/model.xlsx", 2)
sa._getSpectificationsWithInterpolations()

{'column': 22,
 'specification_name': 'Retail price (PU)',
 'unit': 'EUR/unit',
 'interpolation': None,
 'values': [135.0,
  132.22222222222223,
  129.44444444444446,
  126.66666666666667,
  123.88888888888889,
  121.11111111111111,
  118.33333333333334,
  115.55555555555556,
  112.77777777777777,
  110,
  107.22222222222223,
  104.44444444444444,
  101.66666666666667,
  98.88888888888889,
  96.11111111111111,
  93.33333333333334,
  90.55555555555556,
  87.77777777777777,
  85.0,
  82.22222222222223,
  79.44444444444444,
  76.66666666666667]}